## 6. Classification
### 1. Merge one hot encoded genres to lyrics

In [4]:
import pandas as pd

df = pd.read_csv('../dataset/Lyrics_en_artists_clean_train.csv', index_col=['Band'])
df_val = pd.read_csv('../dataset/Lyrics_en_artists_clean_test.csv', index_col=['Band'])
df

,Lyrics,'n,a,abstract,accordion,acid,acoustic,adoracao,adult,african,...,western,wonky,world,worship,wrestling,ye,zapstep,zolo,zouk,zydeco
Band,,,,,,,,,,,,,,,,,,,,,
Exhorder,youve given your life to decadent goals with a...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Heart,Hands on the wheel gridlock and steel dogs are...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Missing Persons,"I feel I was lost, never meant to be found, Fl...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Elmore James,"I feel bad, I feel terrible, Im just as sad as...",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Lumidee,Could never accept reality in which I was pres...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Oasis,I live my life in the city Theres no easy way ...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Carly Simon,The lies that you tell will leave you alone Th...,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
Trina,Da bitch is back I say da bitch is back Yeah d...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Donny Osmond,This is the moment; this is the day When I sen...,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Label exploration  

In [5]:
%matplotlib inline

y_train = df.drop(['Lyrics'], axis=1)
y_test = df_val.drop(['Lyrics'], axis=1)
test_lyrics = df_val['Lyrics']
del df
del df_val
#pd.scatter_matrix(df, alpha = 0.3, figsize = (14,8), diagonal = 'kde');

In [5]:
import seaborn as sns
sns.heatmap(y_train.corr())

ModuleNotFoundError: No module named 'seaborn'

In [ ]:
from sklearn.decomposition import PCA
from IPython import get_ipython
get_ipython().run_line_magic('matplotlib', 'inline')

import matplotlib.pyplot as plt
import matplotlib.cm as cm
import pandas as pd
import numpy as np

pca = PCA()
pca.fit(y_train)

# Dimension indexing
dimensions = dimensions = ['Dimension {}'.format(i) for i in range(1,len(pca.components_)+1)]

# PCA components
components = pd.DataFrame(np.round(pca.components_, 4), columns = list(y_train.keys()))
components.index = dimensions

# PCA explained variance
ratios = pca.explained_variance_ratio_.reshape(len(pca.components_), 1)
variance_ratios = pd.DataFrame(np.round(ratios, 4), columns = ['Explained Variance'])
variance_ratios.index = dimensions

# Create a bar plot visualization
fig, ax = plt.subplots(figsize = (14,8))

# Plot the feature weights as a function of the components
components.plot(ax = ax, kind = 'bar');
ax.set_ylabel("Feature Weights")
ax.set_xticklabels(dimensions, rotation=0)


# Display the explained variance ratios
for i, ev in enumerate(pca.explained_variance_ratio_):
    ax.text(i-0.40, ax.get_ylim()[1] + 0.05, "Explained Variance\n          %.4f"%(ev))

# Return a concatenated DataFrame
pd.concat([variance_ratios, components], axis = 1)

### 3. Doc2Vec Deep learning LSTM classifier

In [6]:
from gensim.models.doc2vec import Doc2Vec

doc2vec = Doc2Vec.load("../dataset/doc2vec")
doc_vectors = doc2vec.wv
print("Number of doc vectors: {}".format(len(doc_vectors.vocab)))

C:\Users\jescobedo\AppData\Local\Continuum\anaconda3\envs\ml\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Number of doc vectors: 61825


In [12]:
from gensim.utils import simple_preprocess
import nltk
from nltk.corpus import stopwords
import spacy
import en_core_web_sm

nltk.download('stopwords')
stop_words = stopwords.words('english')
stop_words.extend(['s', 'chorus'])

# python -m spacy download en
nlp = en_core_web_sm.load(disable=['parser', 'ner'])
allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']
    
def transform_lyric(lyric):
    lyric = simple_preprocess(str(lyric), deacc=True)

    new_lyric = list()
    for word in lyric:
        if(word not in stop_words):
            new_lyric.append(word)
    lyric = new_lyric


    lemma_lyric = list()
    doc = nlp(" ".join(lyric)) 
    lemma_lyric = list()
    for token in doc:
        if(token.pos_ in allowed_postags):
            lemma_lyric.append(token.lemma_)
    lyric = lemma_lyric
    return lyric

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\jescobedo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [15]:
import numpy as np
Text_INPUT_DIM = 100
train_size = y_train.shape[0] 
test_size = y_test.shape[0] 
text_train_arrays = np.zeros((train_size, Text_INPUT_DIM))
text_test_arrays = np.zeros((test_size, Text_INPUT_DIM))

for i in range(train_size):
    text_train_arrays[i] = doc2vec.docvecs[i]
    
for i in range(test_size):
    test_lyrics[i] = transform_lyric(test_lyrics[i])
    text_test_arrays[i] = doc2vec.infer_vector(test_lyrics[i])
#text_test_arrays = doc2vec.infer_vector(test_lyrics)
#del test_lyrics

In [16]:
from keras import backend as K
K.clear_session()

In [24]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding

model_conv = Sequential()
model_conv.add(Embedding(input_dim=train_size, output_dim=100))
model_conv.add(Dropout(0.2))
model_conv.add(Conv1D(64, 5, activation='relu'))
model_conv.add(MaxPooling1D(pool_size=4))
model_conv.add(LSTM(100))
model_conv.add(Dense(y_train.shape[1], activation='sigmoid'))
model_conv.compile(loss='binary_crossentropy', optimizer='adam',    metrics=['accuracy'])
model_conv.summary()

estimator = model_conv.fit(text_train_arrays, y_train, validation_data=(text_test_arrays,y_test), epochs = 2, batch_size=32)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, None, 100)         26693200  
_________________________________________________________________
dropout_6 (Dropout)          (None, None, 100)         0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, None, 64)          32064     
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, None, 64)          0         
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               66000     
_________________________________________________________________
dense_6 (Dense)              (None, 646)               65246     
Total params: 26,856,510
Trainable params: 26,856,510
Non-trainable params: 0
________________________________________________________________

InvalidArgumentError: indices[26,31] = -1 is not in [0, 266932)
	 [[Node: embedding_6/embedding_lookup = GatherV2[Taxis=DT_INT32, Tindices=DT_INT32, Tparams=DT_FLOAT, _class=["loc:@training_5/Adam/Assign_2"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_6/embeddings/read, embedding_6/Cast, training_5/Adam/gradients/embedding_6/embedding_lookup_grad/concat/axis)]]

In [18]:
#Bryce Fox
#Lucy
#genres: "indie poptimism","modern alternative rock","modern rock"

lyric = "So bring your vibe over here here here \
Throw it up in the air air air, oh \
I run my hands through your hair hair hair \
And give you that love cuz you're oh so rare \
You look better over here then take you do over there \
And I don't wanna share \
 \
You don't let a good thing go to waste \
So I took the lipstick off that face \
I don't wanna share, yeah \
 \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
 \
She said she ain't Lucy \
I said not yet \
She said she ain't Lucy \
I'll fix you up something real real tall \
It's looking up, we should take it all off \
Yeah \
You remind me why the stars don't fall \
I swear I'll love you like your last name's Ball \
I'll take you to the lair if we make it past the stairs \
I don't wanna share, yeah \
 \
You don't let a good thing go to waste \
So I took the lipstick off that face \
I don't wanna share, yeah \
 \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
 \
She said she ain't Lucy \
I said not yet \
She said she ain't Lucy \
 \
Said I'm just tryna get to know ya \
So bite your tongue before I bite it for ya \
Yeah \
Said I'm just tryna get to know ya \
So bite your tongue before I bite it for ya \
 \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
If your name was Lucy, I'd put Lucy in her Lucifer \
My god, what in the devil, bring the animal right out of her \
 \
She said she ain't Lucy \
I said not yet \
She said she ain't Lucy \
I said not yet \
(Said I'm just tryna get to know ya) \
(So bite your tongue before I bite it for ya) \
She said she ain't Lucy \
I said not yet \
(Said I'm just tryna get to know ya) \
She said she ain't Lucy \
I said not yet" 

lyric = transform_lyric(lyric)
print(lyric)

['bring', 'vibe', 'throw', 'air', 'air', 'air', 'run', 'hand', 'hair', 'hair', 'hair', 'give', 'love', 'rare', 'look', 'better', 'take', 'wanna', 'share', 'let', 'good', 'thing', 'go', 'waste', 'take', 'lipstick', 'face', 'wanna', 'share', 'name', 'lucy', 'put', 'lucy', 'lucifer', 'devil', 'bring', 'animal', 'right', 'name', 'lucy', 'put', 'lucy', 'lucifer', 'devil', 'bring', 'animal', 'right', 'say', 'lucy', 'say', 'yet', 'say', 'lucy', 'fix', 'something', 'real', 'real', 'tall', 'look', 'take', 'remind', 'star', 'fall', 'swear', 'love', 'last', 'name', 'ball', 'take', 'lair', 'make', 'past', 'stair', 'wanna', 'share', 'let', 'good', 'thing', 'go', 'waste', 'take', 'lipstick', 'face', 'wanna', 'share', 'name', 'lucy', 'put', 'lucy', 'lucifer', 'devil', 'bring', 'animal', 'right', 'name', 'lucy', 'put', 'lucy', 'lucifer', 'devil', 'bring', 'animal', 'right', 'say', 'lucy', 'say', 'yet', 'say', 'lucy', 'say', 'tryna', 'get', 'know', 'bite', 'tongue', 'bite', 'say', 'tryna', 'get', 'know

In [19]:
vector = np.zeros((1, Text_INPUT_DIM))
vector[0] = doc2vec.infer_vector(lyric)
y_pred = model_conv.predict(vector)
idx = (-y_pred[0]).argsort()[:10]
dict(zip(y_train.columns.values[idx],y_pred[0][idx]))

InvalidArgumentError: indices[0,32] = -1 is not in [0, 266932)
	 [[Node: embedding_1/embedding_lookup = GatherV2[Taxis=DT_INT32, Tindices=DT_INT32, Tparams=DT_FLOAT, _class=["loc:@dropout_1/cond/Switch_1"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](embedding_1/embeddings/read, embedding_1/Cast, embedding_1/embedding_lookup/axis)]]